In [2]:
from pytrends.request import TrendReq
from utils import trends 

pytrends = TrendReq(hl='en-US', tz=360)


In [7]:
#  notes:
#  maximum single day duration : 8 months
#  maximum weekly duration     : 62 months
#  maximum months              : 


kw_list = ["S&P 500 Index"]
# Finance: 1163
pytrends.build_payload(kw_list, cat=1163, timeframe='2005-06-14 2020-08-14', geo='', gprop='')
pytrends.interest_over_time()

,S&P 500 Index,isPartial
date,,
2005-07-01,12,False
2005-08-01,7,False
2005-09-01,8,False
2005-10-01,2,False
2005-11-01,18,False
...,...,...
2020-04-01,60,False
2020-05-01,52,False
2020-06-01,62,False


In [5]:
df = trends.pullTrends(pytrends, ["car"], '2015-06-14' ,'2016-08-14')

2016-08-14
2015-06-01 00:00:00
2015-06-30 00:00:00
2015-07-01 00:00:00
2015-07-31 00:00:00
2015-08-01 00:00:00
2015-08-31 00:00:00
2015-09-01 00:00:00
2015-09-30 00:00:00
2015-10-01 00:00:00
2015-10-31 00:00:00
2015-11-01 00:00:00
2015-11-30 00:00:00
2015-12-01 00:00:00
2015-12-31 00:00:00
2016-01-01 00:00:00
2016-01-31 00:00:00
2016-02-01 00:00:00
2016-02-29 00:00:00
2016-03-01 00:00:00
2016-03-31 00:00:00
2016-04-01 00:00:00
2016-04-30 00:00:00
2016-05-01 00:00:00
2016-05-31 00:00:00
2016-06-01 00:00:00
2016-06-30 00:00:00
2016-07-01 00:00:00
2016-07-31 00:00:00


In [158]:
monthly, daily_list = df

print(daily_list[0])
print(monthly)

# for month in daily_list:

            car  isPartial
date                      
2010-06-01   95      False
2010-06-02   93      False
2010-06-03   90      False
2010-06-04   91      False
2010-06-05   92      False
2010-06-06   94      False
2010-06-07   92      False
2010-06-08   97      False
2010-06-09   90      False
2010-06-10   89      False
2010-06-11   87      False
2010-06-12   91      False
2010-06-13   95      False
2010-06-14   88      False
2010-06-15   88      False
2010-06-16   88      False
2010-06-17   88      False
2010-06-18   87      False
2010-06-19   95      False
2010-06-20   96      False
2010-06-21   94      False
2010-06-22   93      False
2010-06-23   88      False
2010-06-24   91      False
2010-06-25   93      False
2010-06-26   95      False
2010-06-27   96      False
2010-06-28  100      False
2010-06-29   97      False
2010-06-30   91      False
            car  isPartial
date                      
2010-07-01   94      False
2010-08-01   91      False
2010-09-01   85      False
2

NameError: name 'X' is not defined

In [77]:
df.index

DatetimeIndex(['2005-07-01', '2005-08-01', '2005-09-01', '2005-10-01',
               '2005-11-01', '2005-12-01', '2006-01-01', '2006-02-01',
               '2006-03-01', '2006-04-01',
               ...
               '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01',
               '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01',
               '2020-07-01', '2020-08-01'],
              dtype='datetime64[ns]', name='date', length=182, freq=None)

odict_keys(['Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 'Jul-15', 'Aug-15', 'Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16'])